In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
import shutil
import numpy as np
import tensorflow as tf
from yolov3.dataset import Dataset

from yolov3.yolov4 import Create_Yolo,Create_Yolo1,compute_loss
from yolov3.yolov4 import compute_loss1
from yolov3.utils import load_yolo_weights
from yolov3.utils import load_yolo_weights, detect_image, image_preprocess, postprocess_boxes, nms, read_class_names

from yolov3.configs import *
from evaluate_mAP import get_mAP
    
from supportFunctions import draw_bbox

from yolov3.dataset import Dataset

testset = Dataset('test')

def read_class_names(class_file_name):
    # loads class name from a file
    names = {}
    with open(class_file_name, 'r') as data:
        for ID, name in enumerate(data):
            names[ID] = name.strip('\n')
    return names
NUM_CLASS = len(read_class_names(TRAIN_CLASSES))
NUM_CLASS

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18342033821482999719
]


21

In [2]:

from yolov3.yolov3 import *
from yoloModify import yolo33
yoloNew=yolo33(input_size=416, channels=3, training=True, CLASSES=TRAIN_CLASSES)


TRAIN_FROM_CHECKPOINT=True

if TRAIN_FROM_CHECKPOINT:
    try:
        print("\n\t load checkpoint!!!")
        yoloNew.load_weights(f"./checkpoints/{TRAIN_MODEL_NAME}")
        print("\n\t loading complete")

    except ValueError:
        print("Shapes are incompatible, transfering Darknet weights")
        TRAIN_FROM_CHECKPOINT = False


optimizer = tf.keras.optimizers.Adam()





**

	 load checkpoint!!!

	 loading complete


In [9]:

phoscLoss=0
NUM_CLASS = len(read_class_names(TRAIN_CLASSES))

def pred_step(nm,counter,image_data, target):
    
    if 1:
    
        pred_result = yoloNew(image_data, training=False)
        #print("\n\t image_data pred:",image_data.shape)
        #print("\n\t len(pred_result) pred:",len(pred_result))
        image2=tf.squeeze(image_data)

        #print("\n\t\t 1.pred_result pred=",pred_result[0].shape)
        #print("\n\t\t 2.pred_result pred=",pred_result[1].shape)
        #print("\n\t\t 3.pred_result pred=",pred_result[2].shape)

        #print("\n\t image2=",image2.shape)
        #pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred_result]
        #print("\n\t pred_bbox =",pred_bbox )
        #pred_bbox = tf.concat(pred_bbox, axis=0)


        # optimizing process
        grid = 3 if not TRAIN_YOLO_TINY else 2
        for i in range(grid):
            """
            if i>0:
                continue
            """
            '''
                these r 3 feature maps of size 52*52,26*26,13*13,

            '''
            conv, pred = pred_result[i*2], pred_result[i*2+1]
            #print("\n\t\t conv pred:",conv.shape)
            #print("\n\t\t pred pred=",pred.shape)


            ''' 
                (xc,yc,w,h,obj,pro) is last dimension of size 6

                pred = (1, 13, 13, 3, 6) is reshaped to 
                pred_bbox = (507, 6)   169 is len of single anchor tot 3 vectors             
                 #2.pred_result: (1, 52, 52, 2325)
                conv: (1, 26, 26, 18)
                pred = (1, 26, 26, 3, 6)
                pred_bbox = (2028, 6)  676 is lenght of single anchor tot 3 vectors

                image_data= (1, 416, 416, 3)
                pred = (1, 52, 52, 3, 6)
                pred_bbox = (8112, 6)  2704 is len of single anchor tot 3 vectors

            '''

            pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred]
            #print("\n\t 1.pred_bbox pred=",len(pred_bbox))

            pred_bbox = tf.concat(pred_bbox, axis=0)

            #print("\n\t 2.pred_bbox pred=",pred_bbox.shape)

            ''' 
            this removes invalid boxes like boxes having low confidence or 
            which has size greater than image

            '''
            #print("\n\t pred_bbox=",len(pred_bbox))
            bboxes = postprocess_boxes(pred_bbox, image2.numpy(), YOLO_INPUT_SIZE,0.25,NUM_CLASS)
            #print("\n\t befor nms bboxes =",bboxes.shape)


            bboxes = nms(bboxes, 0.1, method='nms')


            #print("\n\t after nms 1.bboxes =",len(bboxes))
            #print("\n\t after nms 2.bboxes =",bboxes)

            #draw_bbox() missing 1 required positional argument: 'bboxes
            '''
                 draw_bbox(nm,counter,level, image, bboxes, CLASSES=YOLO_COCO_CLASSES, show_label=True, show_confidence=True,
                  Text_colors=(255, 255, 0), rectangle_colors='', tracking=False)
            '''
            #print("\n\t bboxes len:",len(bboxes))


            if 1:
                try:
                    if bboxes:
                    #print("image2.numpy()",image2.numpy().shape)
                        resultImage = draw_bbox(nm,counter,i,255*image2.numpy(),bboxes,None,CLASSES=TRAIN_CLASSES, rectangle_colors="")
                except Exception as e:
                    pass
            #cv2.imwrite(f'./pred/{counter}.png',resultImage)
            #input("check")



#mAP_model = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=TRAIN_CLASSES) # create second model to measure mAP


In [10]:
best_val_loss = 1000

for counter,(image_data, target,allImagesName) in enumerate(testset):
    nm=str(np.random.randint(10000))+".png"
    nm=str(counter)+".png"

    pred_step(nm,counter,image_data, target)



KeyboardInterrupt: 